<a href="https://colab.research.google.com/github/dominique-stk/Automatizando_envio_de_e-mail/blob/main/Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Passo a passo de construção de código:
- 1: Importar a base de dados 
- 2: calcular o faturamento de cada loja
- 3: calcular a quantida de produtos vendidos de cada loja
- 4: calcular o ticket médio dos produtos de cada loja 
- 5: enviar e-mail 

In [ ]:
#importando base de dados com a biblioteca pandas
import pandas as pd
tabela_vendas = pd.read_excel("Vendas.xlsx")

#Calculando o faturamento de cada loja
tabela_faturamento = tabela_vendas[['ID Loja', 'Valor Final']].groupby("ID Loja").sum()
tabela_faturamento = tabela_faturamento.sort_values(by="Valor Final", ascending=True)

#calculando a quantidade de produtos vendidos 

tabela_quantidade = tabela_vendas[["ID Loja", "Quantidade"]].groupby("ID Loja").sum()
tabela_quantitade = tabela_quantitade.sort_values(by="Quantidade", ascending=True)

#calculando o ticket médio dos produtos de cada loja faturamento/quantidade
ticket_medio = (tabela_faturamento["Valor Final"] / tabela_quantidade["Quantidade"]).to_frame()
ticket_medio = ticket_medio.rename(columns={0: "Ticket Medio"})


#Função enviar e-mail
def enviar_email(nome_loja, tabela):
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f""" 
  <p> Prezados, </p>
  <p> Segue relatório de vendas</p>
  {tabela.to_html()}"""  #editar
    
  msg = email.message.Message()
  msg['Subject'] = f"Relatório de Vendas - {nome_loja}" #editar
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
  # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = '' #editar
  msg['To'] = '' #editar
  password = "" #editar
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')


# enviando_email para  diretoria com todos os dados
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
enviar_email("Diretoria", tabela_completa)


#e-mail para cada loja com filtro de linhas e colunas
lista_lojas = tabela_vendas["ID Loja"].unique()
for loja in lista_lojas:
  tabela_loja = tabela_vendas.loc[tabela_vendas["ID Loja"] == loja, ["ID Loja", "Quantidade", "Valor Final"]]
  tabela_loja = tabela_loja.groupby("ID Loja").sum()
  tabela_loja["Ticket Medio"] = tabela_loja["Valor Final"] / tabela_loja["Quantidade"]
  enviar_email(loja, tabela_loja)






